###### Домашнее задание к лекции "Основы веб-скрапинга"
Обязательная часть

Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.
Дополнительная часть (необязательная)

Функция из обязательной части задания должна быть расширена следующим образом:

    кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
    в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:

<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>



In [ ]:
import pandas as pd

import requests

# метод get
res = requests.get('https://habr.com')
# res
res.status_code

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
# cформируем поисковый запрос, с  выводом по дате
URL = 'https://habr.com/ru/search/?q=python&target_type=posts&order=date'

In [ ]:
# выполним запрос
req = requests.get(URL)



In [ ]:
#  проверка на возврат нормального отвтета от сервера
req.status_code

In [ ]:
req.text

In [ ]:
# создаем объект, через методы которого будем искать нужные теги и извлекать их содержимое
soup = BeautifulSoup(req.text)


In [ ]:
# Заголовок страницы c тегами
print(soup.title)


In [ ]:
# Сам текст заголовка
print(soup.title.text)

In [ ]:
# HTML-код его родителя
print(soup.title.parent)

In [ ]:
# функция find_all позволяет найти все указанные теги с нужными атрибутами (с вложениями), возвращает список
news = soup.find_all('div', class_='posts__item')
print(len(news))
print(news[0])

In [ ]:
# Вывод отформатированной страницы:
print(soup.prettify())

In [ ]:
# функцей find_all  находим контейнеры всех публикаций, возвращаем список
articles = soup.find_all('div', class_='tm-article-snippet tm-article-snippet')
print('Всего на странице:', len(articles))


In [ ]:
# Выводим информацию по первой статье в нормальном виде
print(articles[0].prettify())
print(type(articles[0]))

In [ ]:

# Выделить время
el_time = articles[0].find('span', class_='tm-article-snippet__datetime-published')
time_daytime = el_time.find('time').get('datetime')
print (time_daytime)

# Выделить ссылку на статью
el_ref =  articles[0].find('a', class_='tm-article-snippet__title-link')
# Выделить заголовок
title_ = el_ref.text
print (text_link)
# Скомпоновать полную ссылку
link = 'https://www.habr.com' + el_ref.get('href')
print (link)



In [ ]:
# Обернуть в цикл:

for el in articles:
    # Выделить время
    el_time = el.find('span', class_='tm-article-snippet__datetime-published')
    time_daytime = el_time.find('time').get('datetime')
    print (time_daytime)
    # Выделить ссылку на статью
    el_ref =  el.find('a', class_='tm-article-snippet__title-link')
    # Выделить заголовок
    title_ = el_ref.text
    print (title_)
    # Скомпоновать полную ссылку
    link = 'https://www.habr.com' + el_ref.get('href')
    print (link)
    print()

In [ ]:
# Функция возврата статей по запросу
# Пока с одним запросом
def get_habr_articles(request, order='relevance'):
    
    url = 'https://habr.com/ru/search/?q=' + request + '&target_type=posts&order=' +  order
    print(url)
    req = requests.get(url).text
    soup = BeautifulSoup(req)

    # функцей find_all  находим контейнеры всех публикаций, возвращаем список
    articles = soup.find_all('div', class_='tm-article-snippet tm-article-snippet')
    habr_articles = pd.DataFrame()
    for el in articles:
    # Выделить время
        el_time = el.find('span', class_='tm-article-snippet__datetime-published')
        day_time = el_time.find('time').get('datetime')
    # Выделить ссылку на статью
        el_ref =  el.find('a', class_='tm-article-snippet__title-link')
    # Выделить заголовок
        title_ = el_ref.text
    # Скомпоновать полную ссылку
        link = 'https://www.habr.com' + el_ref.get('href')
        row = {'Дата':  day_time, 'Заголовок': title_, 'Ссылка': link}
        habr_articles = pd.concat([habr_articles, pd.DataFrame([row])]) 
        
    return habr_articles.reset_index(drop=True)
 
# Есть три варианта сортировки:
# по релевантоности - relevance ( параметр по умолчанию)
# по времени - date 
# по рейтингу - rating


df_articles = get_habr_articles('python для анализа данных', 'rating')

df_articles.info()
df_articles.head(20)




In [236]:
import time

# Функция поиска текста и числа лайков
def request_text_and_likes(url_txt):
    # Если ссылка с меткой post, то используй 2 иначе 1
    if 'post' in url_txt:
        class_num = '1'
    else:
        class_num = '2' 
    req = requests.get(url_txt).text
    time.sleep(0.1) # задержка на возможный повторный запрос
    soup_text = BeautifulSoup(req)
   # Ищем количество лайков 
    cl_likes_positiv = "tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive "  
    cl_likes_positiv += "tm-votes-meter__value_appearance-article tm-votes-meter__value_rating"
   # Или дизлайков     
    cl_likes_negotiv = "tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_negative"
    cl_likes_negotiv += " tm-votes-meter__value_appearance-article tm-votes-meter__value_rating"   
    
    likes_ = soup_text.find('span', class_= cl_likes_positiv)
    
    if likes_ != None:
        likes_count = int(likes_.text)
    else:  # а вдруг нет лайков??
        likes_ = soup_text.find('span', class_= cl_likes_negotiv) # тогда считаем дизлайки
        if likes_!= None: 
            likes_count = int(likes_.text)
        else: # а вдруг нет и дизлайков??
            likes_count = 0 
    
   # Пробуем класс форматироваия версии 1 -'article-formatted-body article-formatted-body article-formatted-body_version-1'
    text_ = soup_text.find('div', 
                       class_='article-formatted-body article-formatted-body article-formatted-body_version-1' 
                           )
    form_version = 1
    
    if text_ == None:# Ничего не найдено. Нужент формат версии 2:
        text_ = soup_text.find('div', 
                       class_='article-formatted-body article-formatted-body article-formatted-body_version-2' 
                               ) 
        form_version = 2
        
    print (f'Получен текст по ссылке: {url_txt}. Формат версии: {form_version}. Поставлено лайков/дизлайков: {likes_count}')
    return [text_.text, likes_count]

 


# Функция возврата статей по запросу

def get_habr_articles_list(request_lst, order='relevance', pages=1):
# 
    habr_articles = pd.DataFrame()
    
    # Обернём всё в цикл по списку запросов 
    for request in request_lst:
        url = 'https://habr.com/ru/search/?q=' + str(request) + '&target_type=posts&order=' +  order
        req = requests.get(url).text
        time.sleep(0.1)
        soup = BeautifulSoup(req)

    # функцей find_all  находим контейнеры всех публикаций, возвращаем список
        articles = soup.find_all('div', class_='tm-article-snippet tm-article-snippet')
        for el in articles:
    # Выделить время
            el_time = el.find('span', class_='tm-article-snippet__datetime-published')
            day_time = el_time.find('time').get('datetime')
    # Выделить ссылку на статью
            el_ref =  el.find('a', class_='tm-article-snippet__title-link')
    # Выделить заголовок
            title_ = el_ref.text
    # Скомпоновать полную ссылку
            link = 'https://www.habr.com' + el_ref.get('href')
    # Получить текст поста, новости, записи или новости из блога 
    # компании по ссылке c количеством лайков в виде списка
            text_likes_lst = request_text_and_likes(link)
        
    # Добавить новую строку в датафрейм    
            row = {'Дата':  day_time, 
                   'Заголовок': title_, 
                   'Ссылка': link, 
                   'Текст': text_likes_lst[0], 
                   'Лайки': text_likes_lst[1]
                  }
            habr_articles = pd.concat([habr_articles, pd.DataFrame([row])]) 
    # Получиь уникальные строки на основе столбца ссылок, поскольку в нем содержатся уникальные значения:
    habr_articles = habr_articles.drop_duplicates (subset=['Ссылка'])
    
    return habr_articles.reset_index(drop=True)
     
    
# Есть три варианта сортировки:
# по релевантоности - relevance ( параметр по умолчанию)
# по времени - date 
# по рейтингу - rating

search_list = ['анализ данных', 
               'python', 
               'python для анализа данных',
               'python в аналитике больших данных']

df_articles = get_habr_articles_list( search_list, 'date')

#df_articles.info()

print('\n\nНайдено уникальных', len(df_articles), ' значений по ', len(search_list), ' запросам:' )

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", 6)
pd.set_option("display.width", 1000)
pd.set_option("display.colheader_justify", "center")
pd.set_option("display.precision", 2)

display(df_articles.sort_values(by='Ссылка'))

Получен текст по ссылке: https://www.habr.com/ru/post/707514/. Формат версии: 2. Поставлено лайков/дизлайков: -39
Получен текст по ссылке: https://www.habr.com/ru/post/707496/. Формат версии: 2. Поставлено лайков/дизлайков: 17
Получен текст по ссылке: https://www.habr.com/ru/post/707474/. Формат версии: 2. Поставлено лайков/дизлайков: 20
Получен текст по ссылке: https://www.habr.com/ru/post/707464/. Формат версии: 2. Поставлено лайков/дизлайков: -1
Получен текст по ссылке: https://www.habr.com/ru/news/t/707450/. Формат версии: 2. Поставлено лайков/дизлайков: 10
Получен текст по ссылке: https://www.habr.com/ru/post/707392/. Формат версии: 2. Поставлено лайков/дизлайков: 21
Получен текст по ссылке: https://www.habr.com/ru/news/t/707364/. Формат версии: 2. Поставлено лайков/дизлайков: 24
Получен текст по ссылке: https://www.habr.com/ru/post/707358/. Формат версии: 2. Поставлено лайков/дизлайков: 3
Получен текст по ссылке: https://www.habr.com/ru/company/swd_es/blog/707356/. Формат версии:

Получен текст по ссылке: https://www.habr.com/ru/post/706128/. Формат версии: 2. Поставлено лайков/дизлайков: 11
Получен текст по ссылке: https://www.habr.com/ru/company/glowbyte/blog/705802/. Формат версии: 2. Поставлено лайков/дизлайков: 10
Получен текст по ссылке: https://www.habr.com/ru/post/704938/. Формат версии: 1. Поставлено лайков/дизлайков: 60
Получен текст по ссылке: https://www.habr.com/ru/company/skillfactory/blog/704700/. Формат версии: 1. Поставлено лайков/дизлайков: 0
Получен текст по ссылке: https://www.habr.com/ru/post/704648/. Формат версии: 2. Поставлено лайков/дизлайков: 5
Получен текст по ссылке: https://www.habr.com/ru/post/704000/. Формат версии: 2. Поставлено лайков/дизлайков: 328
Получен текст по ссылке: https://www.habr.com/ru/company/ruvds/blog/702932/. Формат версии: 1. Поставлено лайков/дизлайков: 60
Получен текст по ссылке: https://www.habr.com/ru/company/xakep/blog/702934/. Формат версии: 2. Поставлено лайков/дизлайков: 6
Получен текст по ссылке: https:/

,Дата,Заголовок,Ссылка,Текст,Лайки
37,2022-12-22T12:29:25.000Z,Как мы создавали мобильное приложение «Х5 Недв...,https://www.habr.com/ru/company/X5Tech/blog/70...,"Привет, Хабр! Я Дмитрий Сыровнев, руководитель...",3
21,2022-12-24T10:04:09.000Z,Ещё одна подборка книг по QA,https://www.habr.com/ru/company/alfa/blog/707466/,Квалификационные требования. КТ — 178СТребован...,8
18,2022-12-23T08:00:03.000Z,Эволюция электронного наряда-допуска или как и...,https://www.habr.com/ru/company/eaeconsult/blo...,Только ленивый не слышал сегодня о необходимос...,4
17,2022-12-23T08:55:02.000Z,"DeepFake в условиях современности: что, почему...",https://www.habr.com/ru/company/first/blog/707...,"Дипфейк. Что первое приходит на ум, когда вы с...",6
35,2022-12-22T14:41:27.000Z,"Что делают дата-инженеры, когда данные сами дв...",https://www.habr.com/ru/company/garage8/blog/7...,"Привет, Хабр! Меня зовут Андрей Гончаров, я да...",8
56,2022-12-15T15:25:39.000Z,Практический опыт проектирования систем графов...,https://www.habr.com/ru/company/glowbyte/blog/...,Наши коллеги из группы компаний ГлоуБайт ранее...,10
29,2022-12-23T09:03:24.000Z,В DWH только Python,https://www.habr.com/ru/company/inDrive/blog/7...,"Вопреки названию, мы используем далеко не толь...",10
10,2022-12-23T10:55:40.000Z,ТОП-3 ИБ-событий недели по версии Jet CSIRT,https://www.habr.com/ru/company/jetinfosystems...,Обнаружены новые варианты распространения ВПО ...,0
53,2022-12-15T16:12:50.000Z,Подкаст Данные Люди,https://www.habr.com/ru/company/megafon/news/t...,"Привет, Хабра Пипл!В новом выпуске подкаста ""Д...",0
41,2022-12-21T16:39:35.000Z,Деплоим Machine Learning Workloads используя M...,https://www.habr.com/ru/company/otus/blog/706946/,Автор статьи: Рустем ГалиевIBM Senior DevOps E...,5


In [237]:
df_articles.info()

link_post = 'https://habr.com/ru/post/707222/'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Дата       67 non-null     object
 1   Заголовок  67 non-null     object
 2   Ссылка     67 non-null     object
 3   Текст      67 non-null     object
 4   Лайки      67 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 2.7+ KB


In [187]:
# Функция формирования текста
def request_text(url_txt):
    # Если ссылка с меткой post, то используй 2 иначе 1
    if 'post' in url_txt:
        class_num = '1'
    else:
        class_num = '2' 
    
    req = requests.get(url_txt).text
    time.sleep(0.3) # задержка на возможный повторный запрос
    soup_text = BeautifulSoup(req)
   # Нужный класс для постов - 'article-formatted-body article-formatted-body article-formatted-body_version-1' 
   # Нужный класс для всего КРОМЕ post - 'article-formatted-body article-formatted-body article-formatted-body_version-2'
    text_ = soup_text.find('div', 
                       class_='article-formatted-body article-formatted-body article-formatted-body_version-' + 
                       str(class_num))
    print ('------------------------------Текст по ссылке:', url_txt, '----------------------')
    #print(text_.text)
    print ('------------------------------Конец блока----------------------\n')
    return text_.text

link_post_lst = ['https://habr.com/ru/post/707222/', 
                 'https://habr.com/ru/company/alfa/blog/707466/',
                 'https://habr.com/ru/company/southbridge/news/t/595093/',
                 'https://habr.com/ru/news/t/654707/']

for link in link_post_lst:             
    request_text(link)




------------------------------Текст по ссылке: https://habr.com/ru/post/707222/ ----------------------
------------------------------Конец блока----------------------

------------------------------Текст по ссылке: https://habr.com/ru/company/alfa/blog/707466/ ----------------------
------------------------------Конец блока----------------------

------------------------------Текст по ссылке: https://habr.com/ru/company/southbridge/news/t/595093/ ----------------------
------------------------------Конец блока----------------------

------------------------------Текст по ссылке: https://habr.com/ru/news/t/654707/ ----------------------
------------------------------Конец блока----------------------



In [ ]:
link_company_blogs = 'https://habr.com/ru/company/alfa/blog/707466/'
df_articles.info()

In [ ]:
# Пробуем для блогов компаний 
link_company_blogs = 'https://habr.com/ru/company/alfa/blog/707466/'
req = requests.get(link_company_blogs).text
soup_text = BeautifulSoup(req)

# Нужный класс для команий - 'article-formatted-body article-formatted-body article-formatted-body_version-2'
class_num  = '2' 
text_ = soup_text.find('div', 
                       class_='article-formatted-body article-formatted-body article-formatted-body_version-' + 
                       class_num)
print(text_.text)

In [ ]:
link_company_news = 'https://habr.com/ru/company/southbridge/news/t/595093/'

req = requests.get(link_company_news).text
soup_text = BeautifulSoup(req)

# Нужный класс для новостей компаний - 'article-formatted-body article-formatted-body article-formatted-body_version-2'

text_ = soup_text.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-2')
print(text_.text)

In [205]:
# Просто новость
link_news = 'https://habr.com/ru/company/southbridge/news/t/595093/'
req = requests.get(link_news).text
soup_text = BeautifulSoup(req)

# Нужный класс для новостей - 'article-formatted-body article-formatted-body article-formatted-body_version-2'

text_ = soup_text.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-2')
#print(text_.text)

#Лайки
#<span title="Всего голосов 11: ↑10 и ↓1" 
cl_name = "tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating"
like_ = soup_text.find('span', class_=cl_name)
print(like_)
print(int(like_.text))

<span class="tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating" title="Всего голосов 11: ↑10 и ↓1">+9</span>
9


In [222]:
# Страница результато запроса

link_search_req = 'https://habr.com/ru/search/?q=python&target_type=posts&order=relevance'
req = requests.get(link_company_blogs).text
soup_req = BeautifulSoup(req)
#print(soup_req.prettify())
pages = soup_req.find('pagesCount')
print(pages)
# так просто не работает

None


In [ ]:
import pandas as pd
import requests
import time
from datetime import datetime
from dateutil import parser
from bs4 import BeautifulSoup

# Функция преобразования ISO 8601 строки в дату
def covert_to_date (date_time_str):
    # Можно распознать парсером отметку даты и сразу её вернуть
    return parser.parse(date_time_str).date()




# Функция поиска текста и числа лайков
def request_text_and_likes(url_txt):
    
    req = requests.get(url_txt).text
    time.sleep(0.1) # задержка на возможный повторный запрос
    soup_text = BeautifulSoup(req)
    # Пробуем класс форматироваия версии 1 -'article-formatted-body article-formatted-body article-formatted-body_version-1'
    text_ = soup_text.find('div', 
                       class_='article-formatted-body article-formatted-body article-formatted-body_version-1' 
                           )
    form_version = 1
    
    if text_ == None:# Ничего не найдено. Нужент формат версии 2:
        text_ = soup_text.find('div', 
                       class_='article-formatted-body article-formatted-body article-formatted-body_version-2' 
                               ) 
# В выдаче по дате ссылка на 33 странице запроса "python для анализа данных" на одном из прогонов
# запрос: https://habr.com/ru/search/page33/?q=python для анализа данных&target_type=posts&order=date страница выдачи: 33
# новость https://www.habr.com/ru/company/JetBrains-education/news/t/647151/
# осущетвляет редирект на  https://habr.com/ru/company/JetBrains-education/blog/
# пустую страница компании "JetBrains"
# чтобы это обойти пришлось усложнить код...
        
        if text_ == None: # Опять не сработало??? Значит ссылка не рабочая   
            print (f'В поисковой выдаче  ссылка {url_txt} не адресует к конкретной информации на сайте')
            return []
        else: # сработало на новом формате
            form_version = 2
            # определим лайки
            likes_count = likes_counter(soup_text)
            print (f'Получен текст по ссылке: {url_txt}. Формат версии: {form_version}. Поставлено лайков/дизлайков: {likes_count}')
            return [text_.text, likes_count]
    else:# сработало сразу на формате 1, чего суетиться..
        likes_count = likes_counter(soup_text)
        print (f'Получен текст по ссылке: {url_txt}. Формат версии: {form_version}. Поставлено лайков/дизлайков: {likes_count}')
        return [text_.text, likes_count] 
 
# Функция определения количества  лайков
def likes_counter(soup):
   # Ищем количество лайков 
    cl_likes_positiv = "tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive "  
    cl_likes_positiv += "tm-votes-meter__value_appearance-article tm-votes-meter__value_rating"
   # Или дизлайков     
    cl_likes_negotiv = "tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_negative"
    cl_likes_negotiv += " tm-votes-meter__value_appearance-article tm-votes-meter__value_rating"   
    
    likes_count = 0 # нет ни лайков ни дизлайков
    likes_ = soup.find('span', class_= cl_likes_positiv)
    
    if likes_ != None:
        likes_count = int(likes_.text)
    else:  # а вдруг нет лайков??
        likes_ = soup_text.find('span', class_= cl_likes_negotiv) # тогда считаем дизлайки
        if likes_!= None: 
            likes_count = int(likes_.text)
       
    return likes_count

# Функция возврата статей по запросу

def get_habr_articles_list(request_lst, order='relevance', page_start=1, page_finish=1):
 
    habr_articles = pd.DataFrame()
    
    # Обернём всё в цикл по списку запросов 
    for request in request_lst:
        
        for page in range (page_start, page_finish+1):
            # https://habr.com/ru/search/page1/?q=python&target_type=posts&order=relevance
            url = 'https://habr.com/ru/search/page'+str(page)+'/?q=' + str(request) + '&target_type=posts&order=' +  order
            
            req = requests.get(url)
            time.sleep(0.1)
             # Проверить на успех:
            if req.status_code != 200: # что-то пошло не так, например кончились страницы
                break # выход из цикла
            
            soup = BeautifulSoup(req.text)
            print ('Запрос:', url, 'страница выдачи:', page)
    # функцей find_all  находим контейнеры всех публикаций, возвращаем список
            articles = soup.find_all('div', class_='tm-article-snippet tm-article-snippet')
            for el in articles:
    # Выделить дату и время в виде строки ISO даты и времени
                el_time = el.find('span', class_='tm-article-snippet__datetime-published')
    # Извлечь дату из объекта datetime
                data_ = covert_to_date(el_time.find('time').get('datetime'))
    # Выделить ссылку на статью
                el_ref =  el.find('a', class_='tm-article-snippet__title-link')
    # Выделить заголовок
                title_ = el_ref.text
    # Скомпоновать полную ссылку
                link = 'https://www.habr.com' + el_ref.get('href')
    # Получить текст поста, новости, записи или новости из блога 
    # компании по ссылке c количеством лайков в виде списка
                text_likes_lst = request_text_and_likes(link)
                if text_likes_lst: # если есть реальные данные
                # Добавить новую строку в датафрейм    
                    row = {'Дата':  data_, 
                           'Заголовок': title_, 
                           'Ссылка': link, 
                           'Текст': text_likes_lst[0], 
                           'Лайки': text_likes_lst[1]
                          }
                    habr_articles = pd.concat([habr_articles, pd.DataFrame([row])]) 
                   # habr_articles = habr_articles.append(row, ignore_index=True)
     # Получиь уникальные строки на основе столбца ссылок, поскольку в нем содержатся уникальные значения:
    habr_articles = habr_articles.drop_duplicates (subset=['Ссылка'])
    
    return habr_articles.reset_index(drop=True)
     
    
# Есть три варианта сортировки:
# по релевантоности - relevance ( параметр по умолчанию)
# по времени - date 
# по рейтингу - rating

search_list = ['анализ данных', 
               'python', 
               'python для анализа данных',
               'python в аналитике больших данных']

#search_list = ['python для анализа данных']

max_pages = 55
df_articles = get_habr_articles_list( search_list, 'date', 1, max_pages)

df_articles.info()

print('\n\nНайдено уникальных', len(df_articles), 
      ' значений по ', len(search_list), 
      ' запросам на', max_pages, 'или меньше (см. выдачу) страницах')

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", 6)
pd.set_option("display.width", 1000)
pd.set_option("display.colheader_justify", "center")
pd.set_option("display.precision", 2)

display(df_articles.sort_values(by='Дата', ascending = True))


Запрос: https://habr.com/ru/search/page1/?q=анализ данных&target_type=posts&order=date страница выдачи: 1
Получен текст по ссылке: https://www.habr.com/ru/company/timeweb/blog/706746/. Формат версии: 1. Поставлено лайков/дизлайков: 2
Получен текст по ссылке: https://www.habr.com/ru/post/707514/. Формат версии: 2. Поставлено лайков/дизлайков: 0
Получен текст по ссылке: https://www.habr.com/ru/post/707496/. Формат версии: 2. Поставлено лайков/дизлайков: 17
Получен текст по ссылке: https://www.habr.com/ru/post/707474/. Формат версии: 2. Поставлено лайков/дизлайков: 20
Получен текст по ссылке: https://www.habr.com/ru/post/707464/. Формат версии: 2. Поставлено лайков/дизлайков: 0
Получен текст по ссылке: https://www.habr.com/ru/news/t/707450/. Формат версии: 2. Поставлено лайков/дизлайков: 10
Получен текст по ссылке: https://www.habr.com/ru/post/707392/. Формат версии: 2. Поставлено лайков/дизлайков: 21
Получен текст по ссылке: https://www.habr.com/ru/news/t/707364/. Формат версии: 2. Поста

Получен текст по ссылке: https://www.habr.com/ru/post/702414/. Формат версии: 2. Поставлено лайков/дизлайков: 17
Получен текст по ссылке: https://www.habr.com/ru/company/3rdman/blog/706724/. Формат версии: 2. Поставлено лайков/дизлайков: 6
Получен текст по ссылке: https://www.habr.com/ru/company/otus/blog/706630/. Формат версии: 2. Поставлено лайков/дизлайков: 0
Получен текст по ссылке: https://www.habr.com/ru/company/southbridge/blog/706646/. Формат версии: 2. Поставлено лайков/дизлайков: 14
Получен текст по ссылке: https://www.habr.com/ru/company/ruvds/blog/706164/. Формат версии: 1. Поставлено лайков/дизлайков: 40
Получен текст по ссылке: https://www.habr.com/ru/company/odnoklassniki/blog/705520/. Формат версии: 2. Поставлено лайков/дизлайков: 7
Получен текст по ссылке: https://www.habr.com/ru/post/706624/. Формат версии: 2. Поставлено лайков/дизлайков: 10
Получен текст по ссылке: https://www.habr.com/ru/post/706610/. Формат версии: 2. Поставлено лайков/дизлайков: 5
Получен текст по

Получен текст по ссылке: https://www.habr.com/ru/company/vk/blog/704928/. Формат версии: 1. Поставлено лайков/дизлайков: 9
Получен текст по ссылке: https://www.habr.com/ru/company/rambler_and_co/blog/705820/. Формат версии: 2. Поставлено лайков/дизлайков: 8
Получен текст по ссылке: https://www.habr.com/ru/news/t/705880/. Формат версии: 1. Поставлено лайков/дизлайков: 4
Получен текст по ссылке: https://www.habr.com/ru/company/productivity_inside/blog/705874/. Формат версии: 1. Поставлено лайков/дизлайков: 8
Получен текст по ссылке: https://www.habr.com/ru/company/megafon/news/t/705810/. Формат версии: 2. Поставлено лайков/дизлайков: 0
Получен текст по ссылке: https://www.habr.com/ru/company/glowbyte/blog/705802/. Формат версии: 2. Поставлено лайков/дизлайков: 10
Получен текст по ссылке: https://www.habr.com/ru/company/otus/blog/705796/. Формат версии: 2. Поставлено лайков/дизлайков: 6
Получен текст по ссылке: https://www.habr.com/ru/news/t/705794/. Формат версии: 1. Поставлено лайков/ди

Получен текст по ссылке: https://www.habr.com/ru/company/nlmk/blog/704866/. Формат версии: 1. Поставлено лайков/дизлайков: 78
Получен текст по ссылке: https://www.habr.com/ru/company/kryptonite/blog/704914/. Формат версии: 2. Поставлено лайков/дизлайков: 6
Получен текст по ссылке: https://www.habr.com/ru/news/t/705104/. Формат версии: 1. Поставлено лайков/дизлайков: 5
Получен текст по ссылке: https://www.habr.com/ru/post/705112/. Формат версии: 2. Поставлено лайков/дизлайков: 2
Получен текст по ссылке: https://www.habr.com/ru/news/t/705108/. Формат версии: 2. Поставлено лайков/дизлайков: 4
Получен текст по ссылке: https://www.habr.com/ru/news/t/705082/. Формат версии: 1. Поставлено лайков/дизлайков: 54
Запрос: https://habr.com/ru/search/page11/?q=анализ данных&target_type=posts&order=date страница выдачи: 11
Получен текст по ссылке: https://www.habr.com/ru/post/705068/. Формат версии: 2. Поставлено лайков/дизлайков: 62
Получен текст по ссылке: https://www.habr.com/ru/news/t/705034/. Фо

Получен текст по ссылке: https://www.habr.com/ru/company/first/blog/704330/. Формат версии: 2. Поставлено лайков/дизлайков: 43
Запрос: https://habr.com/ru/search/page14/?q=анализ данных&target_type=posts&order=date страница выдачи: 14
Получен текст по ссылке: https://www.habr.com/ru/news/t/704266/. Формат версии: 1. Поставлено лайков/дизлайков: 1
Получен текст по ссылке: https://www.habr.com/ru/news/t/704264/. Формат версии: 2. Поставлено лайков/дизлайков: 1
Получен текст по ссылке: https://www.habr.com/ru/company/infotecs_official/blog/704260/. Формат версии: 2. Поставлено лайков/дизлайков: 9
Получен текст по ссылке: https://www.habr.com/ru/post/704254/. Формат версии: 2. Поставлено лайков/дизлайков: 2
Получен текст по ссылке: https://www.habr.com/ru/news/t/704214/. Формат версии: 1. Поставлено лайков/дизлайков: 4
Получен текст по ссылке: https://www.habr.com/ru/company/otus/blog/704240/. Формат версии: 2. Поставлено лайков/дизлайков: 3
Получен текст по ссылке: https://www.habr.com/ru

...

Запрос: https://habr.com/ru/search/page27/?q=анализ данных&target_type=posts&order=date страница выдачи: 27
Получен текст по ссылке: https://www.habr.com/ru/company/skillfactory/blog/700260/. Формат версии: 1. Поставлено лайков/дизлайков: 1
Получен текст по ссылке: https://www.habr.com/ru/company/onlinepatent/blog/700306/. Формат версии: 2. Поставлено лайков/дизлайков: 3
Получен текст по ссылке: https://www.habr.com/ru/post/700290/. Формат версии: 2. Поставлено лайков/дизлайков: 3
Получен текст по ссылке: https://www.habr.com/ru/company/vk/blog/699852/. Формат версии: 2. Поставлено лайков/дизлайков: 36
Получен текст по ссылке: https://www.habr.com/ru/company/piter/blog/700282/. Формат версии: 1. Поставлено лайков/дизлайков: 24
Получен текст по ссылке: https://www.habr.com/ru/company/ruvds/blog/699648/. Формат версии: 1. Поставлено лайков/дизлайков: 55
Получен текст по ссылке: https://www.habr.com/ru/company/sberbank/blog/700266/. Формат версии: 2. Поставлено лайков/дизлайков: 14
Получе

Получен текст по ссылке: https://www.habr.com/ru/news/t/699230/. Формат версии: 1. Поставлено лайков/дизлайков: 1
Получен текст по ссылке: https://www.habr.com/ru/post/699254/. Формат версии: 2. Поставлено лайков/дизлайков: 2
Получен текст по ссылке: https://www.habr.com/ru/company/selectel/news/t/699246/. Формат версии: 2. Поставлено лайков/дизлайков: 2
Получен текст по ссылке: https://www.habr.com/ru/company/yandex_praktikum/news/t/699244/. Формат версии: 2. Поставлено лайков/дизлайков: 3
Получен текст по ссылке: https://www.habr.com/ru/company/gaz-is/blog/699238/. Формат версии: 2. Поставлено лайков/дизлайков: 3
Получен текст по ссылке: https://www.habr.com/ru/company/regionsoft/blog/699216/. Формат версии: 2. Поставлено лайков/дизлайков: 21
Получен текст по ссылке: https://www.habr.com/ru/company/otus/blog/699220/. Формат версии: 2. Поставлено лайков/дизлайков: 2
Получен текст по ссылке: https://www.habr.com/ru/post/699094/. Формат версии: 2. Поставлено лайков/дизлайков: 5
Получен 

Получен текст по ссылке: https://www.habr.com/ru/company/skillfactory/blog/697752/. Формат версии: 1. Поставлено лайков/дизлайков: 2
Получен текст по ссылке: https://www.habr.com/ru/news/t/698442/. Формат версии: 2. Поставлено лайков/дизлайков: 5
Получен текст по ссылке: https://www.habr.com/ru/post/698432/. Формат версии: 2. Поставлено лайков/дизлайков: 18
Получен текст по ссылке: https://www.habr.com/ru/post/698406/. Формат версии: 2. Поставлено лайков/дизлайков: 8
Получен текст по ссылке: https://www.habr.com/ru/post/698170/. Формат версии: 2. Поставлено лайков/дизлайков: 26
Получен текст по ссылке: https://www.habr.com/ru/news/t/698386/. Формат версии: 2. Поставлено лайков/дизлайков: 12
Получен текст по ссылке: https://www.habr.com/ru/post/698384/. Формат версии: 2. Поставлено лайков/дизлайков: 32
Получен текст по ссылке: https://www.habr.com/ru/company/ruvds/blog/698090/. Формат версии: 1. Поставлено лайков/дизлайков: 33
Получен текст по ссылке: https://www.habr.com/ru/post/698374

Получен текст по ссылке: https://www.habr.com/ru/post/697446/. Формат версии: 2. Поставлено лайков/дизлайков: 16
Получен текст по ссылке: https://www.habr.com/ru/post/697444/. Формат версии: 2. Поставлено лайков/дизлайков: 12
Получен текст по ссылке: https://www.habr.com/ru/company/inDrive/blog/693768/. Формат версии: 2. Поставлено лайков/дизлайков: 4
Получен текст по ссылке: https://www.habr.com/ru/company/ua-hosting/blog/697260/. Формат версии: 1. Поставлено лайков/дизлайков: 26
Получен текст по ссылке: https://www.habr.com/ru/post/697402/. Формат версии: 2. Поставлено лайков/дизлайков: 9
Запрос: https://habr.com/ru/search/page37/?q=анализ данных&target_type=posts&order=date страница выдачи: 37
Получен текст по ссылке: https://www.habr.com/ru/post/697396/. Формат версии: 2. Поставлено лайков/дизлайков: 131
Получен текст по ссылке: https://www.habr.com/ru/post/697388/. Формат версии: 2. Поставлено лайков/дизлайков: 77
Получен текст по ссылке: https://www.habr.com/ru/company/otus/blog/

Запрос: https://habr.com/ru/search/page40/?q=анализ данных&target_type=posts&order=date страница выдачи: 40
Получен текст по ссылке: https://www.habr.com/ru/company/psb/blog/696542/. Формат версии: 2. Поставлено лайков/дизлайков: 5
Получен текст по ссылке: https://www.habr.com/ru/company/visiology/news/t/696532/. Формат версии: 2. Поставлено лайков/дизлайков: 3
Получен текст по ссылке: https://www.habr.com/ru/post/696504/. Формат версии: 2. Поставлено лайков/дизлайков: 7
Получен текст по ссылке: https://www.habr.com/ru/company/rosbank/blog/696478/. Формат версии: 2. Поставлено лайков/дизлайков: 3
Получен текст по ссылке: https://www.habr.com/ru/post/696422/. Формат версии: 2. Поставлено лайков/дизлайков: 6
Получен текст по ссылке: https://www.habr.com/ru/company/otus/blog/696466/. Формат версии: 2. Поставлено лайков/дизлайков: 4
Получен текст по ссылке: https://www.habr.com/ru/company/vk/blog/695564/. Формат версии: 1. Поставлено лайков/дизлайков: 35
Получен текст по ссылке: https://ww

In [227]:
# В выдаче по дате ссылка на 33 странице запроса "python для анализа данных"
# Запрос: https://habr.com/ru/search/page33/?q=python для анализа данных&target_type=posts&order=date страница выдачи: 33
# новость https://www.habr.com/ru/company/JetBrains-education/news/t/647151/
# осущетвляет редирект на  https://habr.com/ru/company/JetBrains-education/blog/
# пустую страница компании
link_empty = 'https://www.habr.com/ru/company/JetBrains-education/news/t/647151/'    
req = requests.get(link_empty).text
soup_req = BeautifulSoup(req)
print(soup_req.prettify())
    

<!DOCTYPE html>
<html data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D" lang="ru">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width,initial-scale=1.0,viewport-fit=cover,maximum-scale=1,user-scalable=0" name="viewport"/>
  <meta content="unsafe-url" name="referrer"/>
  <title>
   Образовательные проекты JetBrains, Санкт-Петербург / Статьи / Хабр
  </title>
  <style>
   /* cyrillic-ext */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveSxf6TF0.woff2) format('woff2');
      unicode-range: U+0460-052F, U+1C80-1C88, U+20B4, U+2DE0-2DFF, U+A640-A69F, U+FE2E-FE2F;
    }

    /* cyrillic */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveQhf6TF0.woff2) form

In [ ]:
from datetime import datetime
d = datetime.now()
only_date, only_time = d.date(), d.time()
print(only_date)
      
print(only_time)
